# Import Libraries

In [1]:
import urllib.request as ur
import json
import pandas as pd
import numpy as np

# Extract Data from API

In [ ]:
url = 'https://data.yorkopendata.org/api/3/action/datastore_search?resource_id=58f995c1-da44-418f-9977-5ac41c63ccba'  
fileobj = ur.urlopen(url)
#print (fileobj.read())

In [2]:
data = fileobj.read()
json_data = json.loads(data)

# Load Data from Response

In [3]:
json_data_records = json_data['result']['records']
payment_df = pd.DataFrame.from_records(json_data_records)

In [4]:
payment_df.head()

,_id,Organisation_Name,Directorate,Department,Service_Plan,Creditor_Name,Payment_Date,Card_Transaction,Transaction_No,Net_Amount,Irrecoverable_VAT,Subjective_Group,Subjective_Subgroup,Subjective_Detail
0,1,City of York Council,Children Educn & Communities,Childrens Specialist Services,Childrens Social Work Services,Dr Susan Cooper Ltd,2021-06-04T00:00:00,202122CR00000001,,1872.00,,Supplies And Services,Services,Medical Fees
1,2,City of York Council,Children Educn & Communities,Childrens Specialist Services,Childrens Social Work Services,Ofsted,2021-06-04T00:00:00,202122CR00000002,,2059.00,,Supplies And Services,Grants and Subscriptions,Subscriptions
2,3,City of York Council,Children Educn & Communities,Childrens Specialist Services,Childrens Social Work Services,Crombie Wilkinson Solicitors,2021-06-04T00:00:00,202122CR00000003,,105.15,,Supplies And Services,Services,External Solicitors
3,4,City of York Council,Children Educn & Communities,Childrens Specialist Services,Childrens Social Work Services,Crombie Wilkinson Solicitors,2021-06-04T00:00:00,202122CR00000004,,215.00,,Supplies And Services,Services,External Solicitors
4,5,City of York Council,Children Educn & Communities,Childrens Specialist Services,Childrens Social Work Services,Miss Rebecca Musgrove,2021-06-04T00:00:00,202122CR00000005,,350.00,,Supplies And Services,Services,Barristers Fees


In [28]:
payment_df['Department'].unique()

array(['Childrens Specialist Services', 'Transp Highways & Environment',
       'Transport Highways and Environ', 'Housing & Community Safety',
       'Education and Skills', 'Adult Social Care',
       'Adult Services Commissioning', 'Communities and Equalities',
       'Customer Experience & Digital', 'Housing & Public Protection',
       'Office of the Director CEC', 'Director CCS Direct Reports',
       'Legal & Governance', 'Commercial Property'], dtype=object)

# Find All Columns in which NAN value/es is/are present

In [23]:
#replace empty string or white space to NAN - preprocessing
payment_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [25]:
payment_df.columns[payment_df.isnull().any()]

Index(['Transaction_No', 'Irrecoverable_VAT'], dtype='object')

# Create New column based on Existing Colum - Encoding

Implementaion Logic to club Departments using below logic

*   'Transp Highways & Environment', 'Transport Highways and Environ' - **Transportation**
*  'Housing & Community Safety','Housing & Public Protection' - **Housing**

*   'Office of the Director CEC', 'Director CCS Direct Reports' - **Director** 
*   'Adult Social Care',
       'Adult Services Commissioning' - **Adult** 
*   'Communities and Equalities',
       'Customer Experience & Digital', 
       'Legal & Governance', 'Commercial Property' - **Communities**
*   'Childrens Specialist Services', 'Education and Skills' - **Education**









In [55]:
education_substring = ['Childrens Specialist Services', 'Education and Skills']
payment_df['department_encoded']  = np.where(
              payment_df['Department'].str.contains('Transp'),
              'Transportation',
              np.where(
                  payment_df['Department'].str.contains('Housing'),
                  'Housing',
                  np.where(
                    payment_df['Department'].str.contains('Director'),
                    'Director',
                    np.where(
                      payment_df['Department'].str.contains('Adult'),
                      'Adult',
                      np.where(
                              payment_df['Department'].str.contains('|'.join(education_substring)),
                              'Education',
                              'Communities'
                      )
                    )
                  )
                )
              )

# Get Year, Month and Day information from Date Column

In [64]:
#convert payment date to DATETIME type
payment_df['Payment_Date'] = pd.to_datetime(payment_df['Payment_Date'])

In [70]:
#Extract Year, Month, Day
payment_df['Year'] = payment_df['Payment_Date'].dt.year
payment_df['Month'] = payment_df['Payment_Date'].dt.month
payment_df['Day'] = payment_df['Payment_Date'].dt.day

# Let's Set Fiscal Year from March

*   Quarter-1 : March, April, May 

*   Quarter-2 : June, July, August 
*   Quarter-3 : September, October, November 


*   Quarter-4 : December, January, February


You Can set any starting month for Fiscal Year, just replace "March" with desired month





In [82]:
payment_fiscal_info = pd.PeriodIndex(payment_df['Payment_Date'], freq='Q-MAR')

In [83]:
payment_fiscal_info.strftime('Q%q')

Index(['Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1',
       'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q2', 'Q2', 'Q1', 'Q1', 'Q1', 'Q1',
       'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1',
       'Q1', 'Q3', 'Q1', 'Q1', 'Q2', 'Q1', 'Q1', 'Q2', 'Q1', 'Q1', 'Q1', 'Q1',
       'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q2', 'Q1', 'Q1', 'Q1', 'Q2', 'Q1', 'Q2',
       'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2',
       'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2',
       'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2', 'Q2',
       'Q2', 'Q2', 'Q2', 'Q1'],
      dtype='object', name='Payment_Date')

In [84]:
payment_df['fiscal_year'] = payment_fiscal_info.year
payment_df['fiscal_quarter'] = payment_fiscal_info.quarter

# Create a new column using exsiting column data.

Use 'Card_Transaction' column and extract number from it and create new column call 'Card_Transaction_num'

There are multiple ways to do it. Here I have showed two methods.

1. Apply function - can be use to create new column using mutiple columns with complex logics
2. numpy vectorization method - This is very fast but hard to implement every time.



```
# First Way
```



In [99]:
%timeit payment_df['Card_Transaction_num'] = payment_df.apply(lambda x: x['Card_Transaction'].split("CR")[-1],axis=1)

100 loops, best of 5: 2.07 ms per loop




```
# Second Method
```



In [100]:
def get_acc_num(card_trans):
  return card_trans.split("CR")[-1]
np_func_get_accno = np.vectorize(get_acc_num)
%timeit payment_df['Card_Transaction_num'] = np_func_get_accno(payment_df['Card_Transaction'])

1000 loops, best of 5: 244 µs per loop


# In-Line Pandas Operations

Fetch Columns which have Numeric Values

In [107]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
payment_df.select_dtypes(include=numerics)

,_id,Net_Amount,Year,Month,Day,fiscal_year,fiscal_quarter
0,1,1872.00,2021,6,4,2021,1
1,2,2059.00,2021,6,4,2021,1
2,3,105.15,2021,6,4,2021,1
3,4,215.00,2021,6,4,2021,1
4,5,350.00,2021,6,4,2021,1
...,...,...,...,...,...,...,...
95,108,3545.98,2021,7,4,2021,2
96,96,-399.00,2021,7,4,2021,2
97,97,-327.00,2021,7,4,2021,2
98,98,-399.00,2021,7,4,2021,2


## Fetch rows based on Condition on Columns

In [109]:
payment_df[payment_df['Subjective_Group']=='Transport Costs']

,_id,Organisation_Name,Directorate,Department,Service_Plan,Creditor_Name,Payment_Date,Card_Transaction,Transaction_No,Net_Amount,...,Subjective_Group,Subjective_Subgroup,Subjective_Detail,department_encoded,Year,Month,Day,fiscal_year,fiscal_quarter,Card_Transaction_num
18,19,City of York Council,Children Educn & Communities,Education and Skills,School Services & Business Sup,Streamline Taxis (York) Ltd - Large Contracts,2021-09-04,202122CR00000019,NaN,106232.07,...,Transport Costs,Transport Hire,Vehicle Hire,Education,2021,9,4,2021,2,00000019
19,20,City of York Council,Children Educn & Communities,Education and Skills,School Services & Business Sup,Streamline Taxis (York) Ltd - Large Contracts,2021-09-04,202122CR00000020,NaN,122423.57,...,Transport Costs,Transport Hire,Vehicle Hire,Education,2021,9,4,2021,2,00000020


In [110]:
#Fetch specific Card_Transaction based on List values
card_trans_list = ['202122CR00000001','202122CR00000090']
payment_df[payment_df['Card_Transaction'].isin(card_trans_list)]

,_id,Organisation_Name,Directorate,Department,Service_Plan,Creditor_Name,Payment_Date,Card_Transaction,Transaction_No,Net_Amount,...,Subjective_Group,Subjective_Subgroup,Subjective_Detail,department_encoded,Year,Month,Day,fiscal_year,fiscal_quarter,Card_Transaction_num
0,1,City of York Council,Children Educn & Communities,Childrens Specialist Services,Childrens Social Work Services,Dr Susan Cooper Ltd,2021-06-04,202122CR00000001,NaN,1872.00,...,Supplies And Services,Services,Medical Fees,Education,2021,6,4,2021,1,00000001
89,90,City of York Council,Health Housing & Adult So Care,Adult Social Care,Adult Safeguarding,United Response,2021-07-04,202122CR00000090,NaN,749.76,...,Supplies And Services,Services,Day Support,Adult,2021,7,4,2021,2,00000090


In [122]:
payment_df[(payment_df['Subjective_Group']=='Supplies And Services') & (payment_df['Subjective_Subgroup']=='Equip, Furn, Mat') & (payment_df['Month']==7)]

,_id,Organisation_Name,Directorate,Department,Service_Plan,Creditor_Name,Payment_Date,Card_Transaction,Transaction_No,Net_Amount,...,Subjective_Group,Subjective_Subgroup,Subjective_Detail,department_encoded,Year,Month,Day,fiscal_year,fiscal_quarter,Card_Transaction_num
40,41,City of York Council,Children Educn & Communities,Education and Skills,Special Educational Needs,No Isolation Ltd,2021-07-04,202122CR00000041,NaN,-679.0,...,Supplies And Services,"Equip, Furn, Mat",Operational Equipment,Education,2021,7,4,2021,2,00000041
43,44,City of York Council,Children Educn & Communities,Education and Skills,Special Educational Needs,No Isolation Ltd,2021-07-04,202122CR00000044,NaN,679.0,...,Supplies And Services,"Equip, Furn, Mat",Operational Equipment,Education,2021,7,4,2021,2,00000044
96,96,City of York Council,Health Housing & Adult So Care,Housing & Community Safety,Building Maintenance,Jewson Limited,2021-07-04,202122CR00000096,NaN,-399.0,...,Supplies And Services,"Equip, Furn, Mat",Operational Materials (Rechargeable Wks),Housing,2021,7,4,2021,2,00000096
97,97,City of York Council,Health Housing & Adult So Care,Housing & Community Safety,Building Maintenance,Jewson Limited,2021-07-04,202122CR00000097,NaN,-327.0,...,Supplies And Services,"Equip, Furn, Mat",Operational Materials (Rechargeable Wks),Housing,2021,7,4,2021,2,00000097
98,98,City of York Council,Health Housing & Adult So Care,Housing & Community Safety,Building Maintenance,Jewson Limited,2021-07-04,202122CR00000098,NaN,-399.0,...,Supplies And Services,"Equip, Furn, Mat",Operational Materials (Rechargeable Wks),Housing,2021,7,4,2021,2,00000098


In [125]:
#get values of specific column based on condition on other column
payment_df.loc[payment_df['Creditor_Name'] == 'Jewson Limited', 'Net_Amount']

,_id,Organisation_Name,Directorate,Department,Service_Plan,Creditor_Name,Payment_Date,Card_Transaction,Transaction_No,Net_Amount,...,Subjective_Group,Subjective_Subgroup,Subjective_Detail,department_encoded,Year,Month,Day,fiscal_year,fiscal_quarter,Card_Transaction_num
96,96,City of York Council,Health Housing & Adult So Care,Housing & Community Safety,Building Maintenance,Jewson Limited,2021-07-04,202122CR00000096,NaN,-399.00,...,Supplies And Services,"Equip, Furn, Mat",Operational Materials (Rechargeable Wks),Housing,2021,7,4,2021,2,00000096
97,97,City of York Council,Health Housing & Adult So Care,Housing & Community Safety,Building Maintenance,Jewson Limited,2021-07-04,202122CR00000097,NaN,-327.00,...,Supplies And Services,"Equip, Furn, Mat",Operational Materials (Rechargeable Wks),Housing,2021,7,4,2021,2,00000097
98,98,City of York Council,Health Housing & Adult So Care,Housing & Community Safety,Building Maintenance,Jewson Limited,2021-07-04,202122CR00000098,NaN,-399.00,...,Supplies And Services,"Equip, Furn, Mat",Operational Materials (Rechargeable Wks),Housing,2021,7,4,2021,2,00000098
99,99,City of York Council,Health Housing & Adult So Care,Housing & Community Safety,Building Maintenance,Jewson Limited,2021-04-16,202122CR00000099,NaN,829.09,...,Supplies And Services,"Equip, Furn, Mat",Operational Materials (Rechargeable Wks),Housing,2021,4,16,2021,1,00000099


# Operations on Columns

In [ ]:
#get values of specific column based on condition on other column
payment_df.loc[payment_df['Creditor_Name'] == 'Jewson Limited', 'Net_Amount']

96   -399.00
97   -327.00
98   -399.00
99    829.09
Name: Net_Amount, dtype: float64

In [126]:
#Get first value from the returned values
payment_df.loc[payment_df['Creditor_Name'] == 'Jewson Limited', 'Net_Amount'].iloc[0]

-399.0

In [128]:
#Another way to get values of specific column based on condition on other column
payment_df.loc[payment_df['Creditor_Name'] == 'Jewson Limited']['Net_Amount']

96   -399.00
97   -327.00
98   -399.00
99    829.09
Name: Net_Amount, dtype: float64

In [132]:
#convert into Array using "values"
payment_df.loc[payment_df['Creditor_Name'] == 'Jewson Limited', 'Net_Amount'].values

array([-399.  , -327.  , -399.  ,  829.09])